In [ ]:
%reload_ext lammpscn.lammps

In [ ]:
%%lmp_script
# ref: https://doi.org/10.1080/08927022.2017.1313418
# setup problem
units           real
atom_style      atomic
boundary        p p p
dimension    3
boundary     p p p

lattice      fcc 5.3919 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1
region       box prism 0 5 0 5 0 5 0 0 0 
create_box   1 box 
create_atoms 1 box
mass         1 1.0
 
# system contains only atoms 
pair_style      lj/cut     12.00000
pair_coeff      1  1   0.238067 3.405

# use LB combination rules
pair_modify    mix arithmetic
 
pair_modify    shift no
pair_modify    tail yes
 
# cutoff list width, in Ang. (from rc_lj)
neighbor            1.200000 bin
 
# list update, check every timestep after 5 steps
neigh_modify    every 1 delay 5 check yes
 
# groups definition
group Argon.dat type      1
 
# compute thermo info 'a la Newton' to compare
# with td.out values (same units)
variable natoms       equal atoms
variable nt_eb        equal ebond/v_natoms*4.184
variable nt_epl       equal eangle/v_natoms*4.184
variable nt_etor      equal (edihed+eimp)/v_natoms*4.184
variable nt_LJ        equal (evdwl-etail)/v_natoms*4.184
variable nt_ES        equal (ecoul+elong)/v_natoms*4.184
variable nt_Ep        equal pe/v_natoms*4.184
variable nt_E         equal etotal/v_natoms*4.184
variable nt_Pat       equal press*1.01325
variable nt_rho       equal density*1000.
variable nt_V         equal vol
variable nt_T         equal temp
 
variable initstep   equal  step
variable nstep      equal  1000
variable perthermo  equal  100
variable pertraj    equal  100000
variable perpdb     equal  v_nstep+1
# nstep must be a multiple of nevery
variable nevery     equal  10
variable nrepeat    equal  v_nstep/v_nevery
variable nfreq      equal  v_nstep+v_initstep
variable finalstep  equal  v_initstep+v_nstep

# print thermo info variable values
thermo_style custom step v_nt_eb v_nt_epl v_nt_etor  v_nt_LJ v_nt_ES v_nt_Ep v_nt_E  v_nt_Pat v_nt_rho v_nt_V  v_nt_T    pe temp press cella cellb cellc cellalpha cellbeta cellgamma pxx pyy pzz pxy pxz pyz
thermo ${perthermo}

fix fprint all print ${perthermo} "step v_nt_eb v_nt_epl v_nt_etor  v_nt_LJ v_nt_ES v_nt_Ep v_nt_E  v_nt_Pat v_nt_rho v_nt_V  v_nt_T    pe temp press cella cellb cellc cellalpha cellbeta cellgamma pxx pyy pzz pxy pxz pyz" file thermo.dat screen no title "step v_nt_eb v_nt_epl v_nt_etor  v_nt_LJ v_nt_ES v_nt_Ep v_nt_E  v_nt_Pat v_nt_rho v_nt_V  v_nt_T    pe temp press cella cellb cellc cellalpha cellbeta cellgamma pxx pyy pzz pxy pxz pyz"
 
## write a restart file
restart ${finalstep} restart.bin

timestep 1.0
 
# NVE run 
#fix 1 all nve
#run ${nstep}
 
# NPT triclinic
fix 1 all npt  temp  60.00  60.00 100  tri 0.986923  0.986923  1000.0 tchain 10  pchain 10  mtk yes tloop 4 ploop 4
run ${nstep}
